# Making animations of planetary systems!

In the last workbook, we ended by having you each make a plot of the outer planets orbiting around the Sun. So we'll start from there!

In [13]:
import rebound
import numpy as np
import matplotlib.pyplot as plt
import imageio # this is new! We're going to be saving our animations as gifs!

# so we can make plots
%matplotlib inline 
%run tools.ipynb # Some behind-the-scenes tools to simplify some complicated code


Remember we have to redefine the function we made last time that allows us to start a new simulation. 

In [2]:
def start_new_sim(star_mass, star_name): # We have to give our def a unique name, and we can pass parameters to it
    sim = rebound.Simulation()
    sim.G = 4.*pi**2.
    sim.units = ('yr', 'AU', 'Msun')
    sim.add(m=star_mass, hash=star_name)  # Here, we use the variables we passed to the definition to initialize our star
    return sim # We always need to return from our definition; here, we want to pass back our simulation


We'll start with the plot that I asked you to make last time! A secret trick is that Rebound can search for the input parameters of some well known objects, like the planets. It does take a little longer though! Rebound gets the data from [this website](https://ssd.jpl.nasa.gov/horizons.cgi#top). You can try finding objects on this page yourself, if you are interested. Ask me if you need help! 

In [3]:
t_final = 5 # Length of simulation in years
N_frames = 200 # Number of frames in the animation

# Start new simulation - you only get 1 widget per simulation, so if you want a new widget, you need a new simulation
sim = start_new_sim(1.,'Sun') # We call the definition, and we must pass the required elements
sim.add('Earth')
sim.add('Jupiter') 
sim.add('Saturn')
sim.add('Uranus')
sim.add('Neptune')
sim.add('Pluto')
sim.move_to_com()

# And now integrate!
integrate_with_widget(sim, t_final, N_frames, 22)


Searching NASA Horizons for 'Earth'... Found: Earth-Moon Barycenter (3).
Searching NASA Horizons for 'Jupiter'... Found: Jupiter Barycenter (5).
Searching NASA Horizons for 'Saturn'... Found: Saturn Barycenter (6).
Searching NASA Horizons for 'Uranus'... Found: Uranus Barycenter (7).
Searching NASA Horizons for 'Neptune'... Found: Neptune Barycenter (8).
Searching NASA Horizons for 'Pluto'... Found: Pluto Barycenter (9).


Widget(N=7, count=2, height=500.0, orbit_data=b'\xb1\xf8\x0e\xbb\xcfN\xf9;q\xe6\xaa\xb7\xab\xfb{?\x14v\xcb<\xf…

This is a good example of an inclined orbit! Pluto got knocked around a lot by the large outer planets when it was a young dwarf planet, so it's orbit is very different than those of the planets. 

<font color='maroon'>Can you tell if Pluto could ever accidentally run into Neptune?</font>
<br><br><br><br>

### Now that we can run a simulation, let's explore a bit more about _how_ they work. 
This simulation takes small steps forward in time. At each step, the integrator will use the laws of gravity to estimate how the positions and velocities of **every** body in the simulation change. At the next step, it will build from the positions and velocities of the star and planets calculated in the previous step.

To mimic this behavior ourselves,the cell below uses a ```for``` loop to incrementally evolve our simulation forward in time. We achieve this by specifying a time step (```dt```) and the number of steps in our loop (```N_steps```). At each step, we add ```dt``` to the simulation's current time and integrate to this new time. Remember that ```sim.integrate``` takes the end time, which we have assigned to the variable ```new_time```. We also generate an animation frame at each step, which will then be stitched together into an animation similar to the widget above. 

Note that we name our animation ```ani``` - the word ```animation``` belongs to a package we imported, so we can't use it as a variable name.

In [6]:
%%capture 
# This cell produces a lot of unnecessary output, so we'll capture that output instead of printing it out.
# The next code cell will actually display the animation.

dt = 0.1 # Let's go forward a tenth of a year each step
N_steps = 20 # Let's integrate it forward for 20 steps, so 20*0.1 = 2 years

# Start a fresh simulation
sim = start_new_sim(1, 'Sun') # We call the definition, and we must pass the required elements
sim.add(m=3.E-6, a=2., e=0.8, inc=radians(20.), f=radians(45.), hash='Planet')
sim.move_to_com()

# We need to create a figure (fig) and axes (ax) for the animation
fig,ax = plt.subplots(1,1,figsize=(5,5))

# The animation is created from a list of frames
frames = []

for i in range(N_steps):
    
    new_time = sim.t+dt # The time we want to integrate to
    sim.integrate(new_time) # And integrate!
    
    frames.append(make_rebound_frame(ax,sim)) # Add the new frame to the list for animation
       
# Create the animation
ani = animation.ArtistAnimation(fig, frames)


Now let's display our animation! Use the row of control buttons below the animation to adjust the animation speed and move forward/backward through the integration steps.

In [7]:
ani

## Your turn:
<font color='maroon'>Make an animation with 20 frames again, but use the orbits of the outer planets and pluto, like in the first plot of this notebook. *Be warned: the more planets you add, the longer the integration will take! It will take about 30 seconds for this to run.* </font>

<font color='maroon'>Take the following steps to complete this: 
1. change the time steps to be one year long, since the outer planets do not move very far in one year.
2. start a new simulation, with the Sun, Jupiter, Saturn, Uranus, Neptune, and Pluto. Try using Rebound's automatic lookup feature. 
3. Run for ~30 seconds, then run the cell that shows the animation</font>


## Gifs

Alternatively, we can turn this animation into a gif that we can use to show our friends! We want to go through a series of time steps, make a plot for each time step, and save a picture at each step. 

In [24]:
%%capture
# this for loop will print out the image that we're saving. if you'd like to see them, uncomment the line above

dt = 1 # Let's go forward a full year each step
N_steps = 100 # Let's integrate it forward for 20 steps, so 20 years

sim = start_new_sim(1, 'Sun') # We call the definition, and we must pass the required elements
sim.add('Jupiter')
sim.add('Saturn')
sim.add('Uranus')
sim.add('Neptune')
sim.add('Pluto')

# As before, we have to re-center the simulation before evolving it forward in time
sim.move_to_com()

for i in range(N_steps):
    
    new_time = sim.t+dt # The time we want to integrate to
    sim.integrate(new_time) # And integrate!
    
    rebound.OrbitPlot(sim, unitlabel="(AU)", color=True)
    plt.savefig('figs/step_'+str(i)+'.png')
    

If you go back to the tab that says 'Home', you'll see a folder listed that reads 'figs/'. If you open that folder, you'll see a list of a bunch of figures! If you open one, you'll see an image of an orbit we simulated. Now, we have to stitch all of those images into a gif! 

We can do that by running the cell below, which goes through each image and stitches it to the previous one, and then exports the movie as a gif!

In [25]:
images = []
filenames = ['figs/step_'+str(i)+'.png' for i in range(N_steps)]
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('movie.gif', images)

You can find this 'movie.gif' file in the other table where you saw the single image. You can download this image by clicking on the check box, and going to the top and pressing 'Download'. Let me know if you have trouble with this! 

## Your Mission: Simulate An Exoplanet's Orbit

Using your exoplanet, initialize and integrate a Rebound simulation for 10,000 planetary orbits. (For instance, if my planet's period is 1 day, I would integrate it for 10,000 days or 27.4 years. Remember than Rebound must have units in years, solar masses, and AU). How does the orbit change? 

Note that you can find your planet's orbital properties at http://exoplanet.eu/catalog/. Enter in the name of your planet in the Planet Search box just above the table. If the Kepler name doesn't work, use the Alt Name you copied down. Click on the planet name. On the resulting page, you should be able to find, at minimum, the planet mass (in Jupiter masses), the stellar mass (in solar masses), and the planetary semi-major axis. If the other parameters aren't listed, assume they are zero. If an inclination is listed in the catalog, the inclination you should use in your simulation in $90^{\circ} - i_{\textrm{catalog}}$  (the difference comes from observational vs. simulation-based definitions of inclinations). Use $f=0$ radians.

Use the cell below to explicitly write down an outline of the code you want to write. Be sure to give yourself notes about all the small steps you need to take, including considering how you're going to display the results.